In [2]:
import sys

sys.path.append('../../')
import json
import torchaudio

import librosa
import torch
from academicodec.models.hificodec.vqvae import VQVAE
from librosa.util import normalize

In [3]:
config_path = './config_24k_320d.json'
with open(config_path, 'r') as f:
    config = json.load(f)
    sample_rate = config['sampling_rate']

In [4]:
print("Init model and load weights")
model = VQVAE(
    config_path,
    ckpt_path="",
    with_encoder=True)
model.cuda()
model.eval()
print("Model ready")

Init model and load weights


/home/ubuntu/miniconda3/envs/tts2/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Model ready


In [5]:
wav_path = './sample.wav'

wav, sr = librosa.load(wav_path, sr=sample_rate)
print("wav.shape:",wav.shape)
assert sr == sample_rate

wav = normalize(wav) * 0.95
wav = torch.FloatTensor(wav).unsqueeze(0)
wav = wav.to(torch.device('cuda'))

wav.shape: (240000,)


In [6]:
acoustic_token = model.encode(wav)

print("acoustic_token:", "\n",acoustic_token)
print("acoustic_token.shape:",acoustic_token.shape)
print("acoustic_token.dtype:",acoustic_token.dtype)

acoustic_token: 
 tensor([[[ 61, 837, 643,  ...,   7, 858, 806],
         [612, 837, 444,  ...,   7, 999, 806],
         [612, 837, 333,  ..., 784, 147,  85],
         ...,
         [128, 837, 737,  ..., 753, 950,  85],
         [986, 837, 737,  ...,  54, 183, 806],
         [957, 162, 292,  ..., 224, 183, 361]]], device='cuda:0')
acoustic_token.shape: torch.Size([1, 750, 8])
acoustic_token.dtype: torch.int64


In [18]:
quant_vector = model.encoder(wav.unsqueeze(1))
print(quant_vector.shape)
y_hat = model.generator(quant_vector)
print(y_hat.shape)

torch.Size([1, 128, 750])
torch.Size([1, 1, 240000])


In [19]:
y_hat = y_hat.squeeze(0)
y_hat = y_hat.cpu()
print(y_hat.size())
torchaudio.save("y_hat.wav", torch.tensor(y_hat), sample_rate, channels_first=True)

torch.Size([1, 240000])


/tmp/ipykernel_1805445/3900361264.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torchaudio.save("y_hat.wav", torch.tensor(y_hat), sample_rate, channels_first=True)


: 